In [1]:
if 'google.colab' in str(get_ipython()):
    !pip install git+https://github.com/ourownstory/neural_prophet.git # may take a while
    #!pip install neuralprophet # much faster, but may not have the latest upgrades/bugfixes
    data_location = "https://raw.githubusercontent.com/ourownstory/neural_prophet/master/"
else:
    data_location = "../"

In [2]:
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level
# set_log_level("ERROR")

In [3]:
df = pd.read_csv(data_location + "example_data/air_passengers.csv")

In [4]:
folds = NeuralProphet().crossvalidation_split_df(df, freq="MS", k=5, fold_pct=0.1, fold_overlap_pct=0.0)
train, val = [], []
for df_train, df_val in folds:
    m = NeuralProphet(
        seasonality_mode="multiplicative", 
#         learning_rate=0.1,
    )
    metrics_train = m.fit(df_train, freq="MS")
    metrics_val = m.test(df_val)
    train.append(metrics_train["MAE"].values[-1])
    val.append(metrics_val["MAE"].values[-1])
#     future = m.make_future_dataframe(df_train, periods=len(df_val), n_historic_predictions=len(df_train))
#     forecast = m.predict(future)
#     fig = m.plot(forecast)
print(train)
print(val)
print(sum(train)/len(train))
print(sum(val)/len(val))

INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 360


INFO - (NP.forecaster._lr_range_test) - lr-range-test results: steep: 2.44E-02, min: 8.21E-01, chosen: 1.41E-01
INFO - (NP.forecaster._lr_range_test) - learning rate range test found optimal lr: 1.41E-01
Epoch[360/360]: 100%|██████████| 360/360 [00:02<00:00, 165.60it/s, SmoothL1Loss=0.000756, MAE=6.01, RegLoss=0]
INFO - (NP.forecaster._evaluate) - Validation metrics:    SmoothL1Loss    MAE
1         0.012 28.272
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 336


INFO - (NP.forecaster._lr_range_test) - lr-range-test results: steep: 1.28E-02, min: 5.17E-01, chosen: 8.12E-02
INFO - (NP.forecaster._lr_range_test) - learning rate range test found optimal lr: 8.12E-02
Epoch[336/336]: 100%|██████████| 336/336 [00:02<00:00, 134.60it/s, SmoothL1Loss=0.000447, MAE=6.11, RegLoss=0]
INFO - (NP.forecaster._evaluate) - Validation metrics:    SmoothL1Loss    MAE
1         0.000  5.426
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 317


INFO - (NP.forecaster._lr_range_test) - lr-range-test results: steep: 1.28E-02, min: 9.88E-01, chosen: 1.07E-01
INFO - (NP.forecaster._lr_range_test) - learning rate range test found optimal lr: 1.07E-01
Epoch[317/317]: 100%|██████████| 317/317 [00:02<00:00, 125.55it/s, SmoothL1Loss=0.000266, MAE=5.59, RegLoss=0]
INFO - (NP.forecaster._evaluate) - Validation metrics:    SmoothL1Loss    MAE
1         0.005 24.557
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 301


INFO - (NP.forecaster._lr_range_test) - lr-range-test results: steep: 1.40E-02, min: 8.21E-01, chosen: 1.07E-01
INFO - (NP.forecaster._lr_range_test) - learning rate range test found optimal lr: 1.07E-01
Epoch[301/301]: 100%|██████████| 301/301 [00:02<00:00, 111.26it/s, SmoothL1Loss=0.000249, MAE=7.24, RegLoss=0]
INFO - (NP.forecaster._evaluate) - Validation metrics:    SmoothL1Loss    MAE
1         0.001 11.806
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 17
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 288


INFO - (NP.forecaster._lr_range_test) - lr-range-test results: steep: 4.66E-02, min: 6.22E-01, chosen: 1.70E-01
INFO - (NP.forecaster._lr_range_test) - learning rate range test found optimal lr: 1.70E-01
Epoch[288/288]: 100%|██████████| 288/288 [00:02<00:00, 110.09it/s, SmoothL1Loss=0.000175, MAE=6.6, RegLoss=0] 
INFO - (NP.forecaster._evaluate) - Validation metrics:    SmoothL1Loss    MAE
1         0.002 29.614


[6.0101911183950065, 6.114388812672008, 5.594424822751214, 7.235396681160762, 6.601710062760573]
[28.271963119506836, 5.4255547523498535, 24.557321548461914, 11.806401252746582, 29.61379051208496]
6.311222299547913
19.93500623703003


In [5]:
folds = NeuralProphet().crossvalidation_split_df(df, freq="MS", k=5, fold_pct=0.1, fold_overlap_pct=0.0)
train, val = [], []
for df_train, df_val in folds:
    m = NeuralProphet(
        seasonality_mode="multiplicative", 
#         learning_rate=0.1,
    )
    metrics_train = m.fit(df_train, freq="MS")
    metrics_val = m.test(df_val)
    train.append(metrics_train["MAE"].values[-1])
    val.append(metrics_val["MAE"].values[-1])
#     future = m.make_future_dataframe(df_train, periods=len(df_val), n_historic_predictions=len(df_train))
#     forecast = m.predict(future)
#     fig = m.plot(forecast)
print(train)
print(val)
print(sum(train)/len(train))
print(sum(val)/len(val))

INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 360


INFO - (NP.forecaster._lr_range_test) - lr-range-test results: steep: 4.66E-02, min: 1.19E+00, chosen: 2.25E-01
INFO - (NP.forecaster._lr_range_test) - learning rate range test found optimal lr: 2.25E-01
Epoch[360/360]: 100%|██████████| 360/360 [00:02<00:00, 159.44it/s, SmoothL1Loss=0.000752, MAE=5.99, RegLoss=0]
INFO - (NP.forecaster._evaluate) - Validation metrics:    SmoothL1Loss    MAE
1         0.012 28.198
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 336


INFO - (NP.forecaster._lr_range_test) - lr-range-test results: steep: 5.11E-02, min: 8.21E-01, chosen: 2.05E-01
INFO - (NP.forecaster._lr_range_test) - learning rate range test found optimal lr: 2.05E-01
Epoch[336/336]: 100%|██████████| 336/336 [00:02<00:00, 143.44it/s, SmoothL1Loss=0.000441, MAE=6.02, RegLoss=0]
INFO - (NP.forecaster._evaluate) - Validation metrics:    SmoothL1Loss    MAE
1         0.000  5.466
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 317


INFO - (NP.forecaster._lr_range_test) - lr-range-test results: steep: 2.44E-02, min: 9.88E-01, chosen: 1.55E-01
INFO - (NP.forecaster._lr_range_test) - learning rate range test found optimal lr: 1.55E-01
Epoch[317/317]: 100%|██████████| 317/317 [00:02<00:00, 125.37it/s, SmoothL1Loss=0.000266, MAE=5.57, RegLoss=0]
INFO - (NP.forecaster._evaluate) - Validation metrics:    SmoothL1Loss    MAE
1         0.005 24.670
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 301


INFO - (NP.forecaster._lr_range_test) - lr-range-test results: steep: 6.15E-02, min: 4.30E-01, chosen: 1.55E-01
INFO - (NP.forecaster._lr_range_test) - learning rate range test found optimal lr: 1.55E-01
Epoch[301/301]: 100%|██████████| 301/301 [00:02<00:00, 104.52it/s, SmoothL1Loss=0.000249, MAE=7.29, RegLoss=0]
INFO - (NP.forecaster._evaluate) - Validation metrics:    SmoothL1Loss    MAE
1         0.001 11.532
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 17
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 288


INFO - (NP.forecaster._lr_range_test) - lr-range-test results: steep: 3.53E-02, min: 1.08E+00, chosen: 1.87E-01
INFO - (NP.forecaster._lr_range_test) - learning rate range test found optimal lr: 1.87E-01
Epoch[288/288]: 100%|██████████| 288/288 [00:02<00:00, 106.98it/s, SmoothL1Loss=0.000175, MAE=6.61, RegLoss=0]
INFO - (NP.forecaster._evaluate) - Validation metrics:    SmoothL1Loss    MAE
1         0.002 29.720


[5.98787463677896, 6.017117933793501, 5.569710306092804, 7.293258338138975, 6.61451908991887]
[28.198434829711914, 5.465634822845459, 24.670042037963867, 11.532272338867188, 29.720279693603516]
6.296496060944622
19.91733274459839


In [6]:
# folds = NeuralProphet().crossvalidation_split_df(df, freq="MS", k=5, fold_pct=0.1, fold_overlap_pct=0.0)
# train, val = [], []
# for df_train, df_val in folds:
#     m = NeuralProphet(
#         seasonality_mode="multiplicative", 
# #         learning_rate=0.1,
# #         normalize="minmax",
#     )
#     metrics_train = m.fit(df_train, freq="MS")
#     metrics_val = m.test(df_val)
#     train.append(metrics_train["MAE"].values[-1])
#     val.append(metrics_val["MAE"].values[-1])
#     future = m.make_future_dataframe(df_train, periods=len(df_val), n_historic_predictions=len(df_train))
#     forecast = m.predict(future)
#     fig = m.plot(forecast)
#     fig.show()
# print(train)
# print(val)
# print(sum(train)/len(train))
# print(sum(val)/len(val))